## Programically Read Ground Truth Data

In [ ]:
import requests
import yaml
from random import random, randint

import cartopy.feature as cfeature
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from shapely.geometry import Polygon

In [ ]:
with open('./config.yml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
def open_geojson(url):
    headers = {'cookie': f'_oauth2_proxy={config["AUTH_TOKEN"]}'}
    res = requests.get(url, headers=headers, params={'limit':500})
    gdf = gpd.read_file(res.text)
    return gdf

## Open GT Collection using GeoPandas

Open source Geospatial data frames library

https://geopandas.readthedocs.io/en/latest/

Allows for easy indexing, manipulation, grouping of data. Assists data subsetting and validation.

In [ ]:
geojson_url = 'https://gt.data.geoanalytics.ca/collections/SE_AGR/items?f=json'
gdf = open_geojson(geojson_url)

In [ ]:
gdf

## Example filter

`Nvx1 == AGR and Nvx3 == FOI1`

In [ ]:
relevant_data = gdf[(gdf['Nvx1'] == 'AGR') & (gdf['Nvx3'] == 'FOI1')]
relevant_data

In [ ]:
gdf

## Example usage of data

- Data comparison
- Plot filtered items

### Compare synthetic data set with ground truth

Example workflow for data validation

In [ ]:
def create_synthetic_polygon(ll_lat, ll_lon, ul_lat, ul_lon):
    return Polygon(((ll_lon, ll_lat), (ll_lon, ul_lat), (ul_lon, ul_lat), (ul_lon, ll_lat)))

In [ ]:
## generate synthetic data

nvx3_values = gdf['Nvx3'].unique()
lat_max = 46.5
lat_min = 46.0
lon_max = -75.0
lon_min = -73.0


synthetic_data = []
for i in range(500):
    m = randint(0, len(nvx3_values)-1)
    nvx3 = nvx3_values[m]
    ll_lat = lat_min + (lat_max - lat_min) * random()
    ll_lon = lon_min + (lon_max - lon_min) * random()
    ul_lat = ll_lat + 0.05
    ul_lon = ll_lon + 0.05
    geom = create_synthetic_polygon(ll_lat, ll_lon, ul_lat, ul_lon)
    synthetic_data.append({'syn_id': i, 'Nvx3': nvx3, 'geometry': geom})

syn_df = gpd.GeoDataFrame(synthetic_data, crs='epsg:4326')
syn_df.head()

In [ ]:
## intersect GT data with synthetic data
intersected_points = gpd.overlay(syn_df, gdf, how='intersection')
intersected_points

In [ ]:
# identify points that match with ground truth
intersected_points[intersected_points['Nvx3_1'] == intersected_points['Nvx3_2']]

In [ ]:
## identify points that do not match with ground truth
intersected_points[intersected_points['Nvx3_1'] != intersected_points['Nvx3_2']]

### Plot relevant points

Example workflow for publishing data

In [ ]:
# https://spatialreference.org/ref/epsg/32662/
plate_carree_epsg = 32662
proj = ccrs.epsg(plate_carree_epsg)
relevant_data_epsg = relevant_data.to_crs(epsg=plate_carree_epsg)

fig, ax = plt.subplots(1,1, subplot_kw={'projection': proj}, figsize=(10, 10))
ax.set_extent([-74.5, -72.5, 46, 44.5], crs=ccrs.PlateCarree())
#ax.set_extent([-77, -69, 48, 42], crs=proj)
ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none'))
ax.add_feature(cfeature.LAKES.with_scale('10m'), alpha=0.5)
ax.add_feature(cfeature.RIVERS.with_scale('10m'), alpha=0.5)
ax.add_feature(cfeature.STATES.with_scale('10m'))
ax.add_geometries(relevant_data_epsg['geometry'], crs=proj, edgecolor='red', linewidth=3)
ax.set_title('Attributes with Nvx1=AGR, Nvx3=FOI1')
fig.show()